# Get Started With nGQL Jupyter Magic Extension

## How To Install nGQL in Jupyter or iPython
First, install nGQL Magic with pip:
```bash
!pip install ipython-ngql
```
Second, load extension:
```bash
$load_ext ngql
```

In [1]:
%load_ext ngql

## How to Use
Connect to Nebula Graph with:

```bash
%ngql --address <ip> --port <port> --user <username> --password <password>
```

In [2]:
%ngql --address 127.0.0.1 --port 9669 --user user --password password

Connection Pool Created


,Name
0,pokemon_club


Make a query after `Connection Pool Created` shown in the connection from last step:


Option 1, it supports one line query as:
```ngql
%ngql <query_line>;
```
Option 2, it also supports multiple queries:

```ngql
%%ngql
<line 0>;
<line 1>;
```

> Note, we will support a 3rd option to enable runing lines from `.ngql` file soon.

In [3]:
%ngql GO FROM "Tom" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id | GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name

,Trainer_Name
0,Tom
1,Jerry
2,Sue
3,Tom
4,Wey


In [4]:
%%ngql
SHOW TAGS;
SHOW HOSTS;

,Host,Port,Status,Leader count,Leader distribution,Partition distribution
0,storaged0,9779.0,ONLINE,0,No valid partition,No valid partition
1,storaged1,9779.0,ONLINE,1,pokemon_club:1,pokemon_club:1
2,storaged2,9779.0,ONLINE,0,No valid partition,No valid partition
3,Total,NaN,None,1,pokemon_club:1,pokemon_club:1


> It's quite easy, right?
You could get help by `%ngql help` for some advanced usage:

```ngql
%ngql help
```

### Using Variables in Query String

We used Jinja2(https://jinja.palletsprojects.com/) as templating method for variables in query string:

```python
trainer = "Sue"
```

```ngql
%%ngql
GO FROM "{{ trainer }}" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id | GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name;
```

In [5]:
trainer = "Sue"

In [6]:
%%ngql
GO FROM "{{ trainer }}" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id | GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name;

,Trainer_Name
0,Jerry
1,Sue
2,Tom
3,Wey


### Using Raw thrift data type as result
By default the result `ngql_result_style` is `pandas`, this enabled us to have table view renderred by Jupyter Notebook.

While, if you would like to get raw results from `neutron2-python` itself, just configure it as below on the fly:

```
%config IPythonNGQL.ngql_result_style="raw"
```

And after querying, the result will be stored in `_`, plesae then refer it to a new variable for further ad-hoc tweaking on it like:
```
$ngql <query>;

result = _

dir(result)
```

In [7]:
%config IPythonNGQL.ngql_result_style="raw"

In [8]:
%%ngql USE pokemon_club;
GO FROM "Tom" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id 
| GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name;

ResultSet(ExecutionResponse(
    error_code=0,
    latency_in_us=6886,
    data=DataSet(
        column_names=[b'Trainer_Name'],
        rows=[Row(
            values=[Value(
                sVal=b'Tom')]),
        Row(
            values=[Value(
                sVal=b'Jerry')]),
        Row(
            values=[Value(
                sVal=b'Sue')]),
        Row(
            values=[Value(
                sVal=b'Tom')]),
        Row(
            values=[Value(
                sVal=b'Wey')])]),
    space_name=b'pokemon_club'))

In [9]:
r = _

In [10]:
r.column_values(key='Trainer_Name')[0]._value.value

b'Tom'

> Change back to `pandas` `ngql_result_style`

In [11]:
%config IPythonNGQL.ngql_result_style="pandas"

In [12]:
%%ngql
GO FROM "Tom" OVER owns_pokemon YIELD owns_pokemon._dst as pokemon_id 
| GO FROM $-.pokemon_id OVER owns_pokemon REVERSELY YIELD owns_pokemon._dst AS Trainer_Name;

,Trainer_Name
0,Tom
1,Jerry
2,Sue
3,Tom
4,Wey


## Only one takeaway: `% ngql help`!
All you have to remember is to use `$ngql help` to have all hints :-)

In [13]:
%ngql help



        Supported Configurations:
        ------------------------
        
        > How to config ngql_result_style in "raw", "pandas"
        %config IPythonNGQL.ngql_result_style="raw"
        %config IPythonNGQL.ngql_result_style="pandas"

        > How to config ngql_verbose in True, False
        %config IPythonNGQL.ngql_verbose=True

        > How to config max_connection_pool_size
        %config IPythonNGQL.max_connection_pool_size=10

        Quick Start:
        -----------

        > Connect to Neubla Graph
        %ngql --address 127.0.0.1 --port 9669 --user user --password password

        > Use Space
        %ngql USE nba

        > Query
        %ngql SHOW TAGS;

        > Multile Queries
        %%ngql
        SHOW TAGS;
        SHOW HOSTS;

        Reload ngql Magic
        %reload_ext ngql

        > Variables in query, we are using Jinja2 here
        name = "nba"
        %ngql USE "{{ name }}"


        
